#Protein Secondary Structure Prediction
Using a Hidden Markov Model to Classify Amino Acid Sequences into Secondary Structure states in order to understand protein folding
#Names
Anila Chundi, Ranjini Ramesh, Shalini Sivakumar



In [78]:
#Install libraries

! pip install nose
! pip install numpy
! pip install pandas
! pip install matplotlib


#CB513 Dataset
- The CB513 dataset is a benchmark dataset widely used for training and testing protein secondary structure prediction models. It contains 513 protein sequences with annotated secondary structures.
- dataset is from here: https://huggingface.co/datasets/proteinea/secondary_structure_prediction/blob/main/CB513.csv

#DSSP Dataset
- The DSSP (Dictionary of Secondary Structure in Proteins) dataset provides detailed secondary structure assignments for protein entries in the Protein Data Bank (PDB).


In [79]:
#Download data here
import requests

url = 'https://huggingface.co/datasets/proteinea/secondary_structure_prediction/resolve/main/CB513.csv'
output_file = 'CB513.csv'

response = requests.get(url)
if response.status_code == 200:
    with open(output_file, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully as {output_file}")
else:
    print(f"Failed to download file. HTTP Status Code: {response.status_code}")

File downloaded successfully as CB513.csv


In [80]:
import copy
import sys

import nose.tools as nt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Data Parsing
- read and understand data in datasets

**CB513_data**
- "input" column contains the amino acid sequences for each protein
  - each row represents a protein sequence as a long string of one-letter amino acid codes
- "dssp3" column represents the secondary structure annotation for the corresponding protein in "3-class DSSP" format
  - H: Helix, E: Strand(beta sheet), C: Coil
  - each char corresponds to the secondary structure of the amino acid in the input column at the same position
- "dssp8" column represents the secondary structure annotation for the corresponding protein in "8-class DSSP" format
  - H: Alpha Helix, E: Extended Strand, C: Coil, G: 3/10 helix, I: π-helix, B: Isolated β-bridge, T: Turn, S: Bend
  - each char corresponds to the secondary structure of the amino acid in the input column at the same position
- "disorder" column indicates a region without a stable secondary structure
  - values: 1 = Disordered, 0 = Ordered
- "cb513_mask" column serves as a mask used to exclude padding, invalid data, etc
  - 1 = valid, 0 = invalid

In [81]:
# Load the CSV file into a DataFrame
cb513_data = pd.read_csv('CB513.csv')

print(cb513_data.head())


                                               input  \
0  RTDCYGNVNRIDTTGASCKTAKPEGLSYCGVSASKKIAERDLQAMD...   
1  GKITFYEDRGFQGRHYECSSDHSNLQPYFSRCNSIRVDSGCWMLYE...   
2  MFKVYGYDSNIHKCVYCDNAKRLLTVKKQPFEFINIMPEKGVFDDE...   
3  APAFSVSPASGASDGQSVSVSVAAAGETYYIAQCAPVGGQDACNPA...   
4  TPAFNKPKVELHVHLDGAIKPETILYFGKKRGIALPADTVEELRNI...   

                                               dssp3  \
0  CCCCCCCHHHCCCCCECHHHHCCCCCCCCEHHHHHHHHHHCHHHHH...   
1  CEEEEEEECCCEEEEEEECCCECCCCCCCCCCCEEEEEECEEEEEC...   
2  CEEEEECCCCCCCCHHHHHHHHHHHHCCCCEEEEECCCECCECCHH...   
3  CCEEEEECCCCCCCCCEEEEEEECCCCEEEEEEECEECCEECCCCC...   
4  CCCCCCCEEEEEEEHHHCCCHHHHHHHHHHHCCCCCCCCHHHHHHH...   

                                               dssp8  \
0  CCCTTCCGGGSCCCCBCHHHHTTTTCSCCBHHHHHHHHHHTHHHHH...   
1  CEEEEEEETTTEEEEEEECSCBSCCTTTCSCCSEEEEEESEEEEES...   
2  CEEEEECCTTTSCCHHHHHHHHHHHHTTCCEEEEESCSBTTBCCHH...   
3  CCEEEEECCSSCCSSCEEEEEEESCCSEEEEEEECEETTEECCCTT...   
4  CCSCCSCEEEEEEEGGGSCCHHHHHHHHHHHTCCCSCSSHHHH

#HMM Training
- Use previous dataset to train the HMM model
  - Use input column as the 'observed states'
  - Use dssp3 as hidden states

In [85]:
observed_states = cb513_data['input'].tolist()
hidden_states = cb513_data['dssp3'].tolist()


for obs, hid in zip(observed_states, hidden_states):
  assert(len(obs) == len(hid))

hmm_data = list(zip(observed_states, hidden_states))
print('--------------------------------')

for i in range(len(hmm_data)):
  print(len(hmm_data[i][0]))
  print(len(hmm_data[i][1]))

'''
hmm_data is a list of pairs, where pair[0] is the observed sequence and pair[1] is the hidden sequence
'''


['RTDCYGNVNRIDTTGASCKTAKPEGLSYCGVSASKKIAERDLQAMDRYKTIIKKVGEKLCVEPAVIAGIISRESHAGKVLKNGWGDRGNGFGLMQVDKRSHKPQGTWNGEVHITQGTTILINFIKTIQKKFPSWTKDQQLKGGISAYNAGAGNVRSYARMDIGTTHDDYANDVVARAQYYKQHGY', 'GKITFYEDRGFQGRHYECSSDHSNLQPYFSRCNSIRVDSGCWMLYEQPNFQGPQYFLRRGDYPDYQQWMGLNDSIRSCRLIPHTGSHRLRIYEREDYRGQMVEITEDCSSLHDRFHFSEIHSFNVLEGWWVLYEMTNYRGRQYLLRPGDYRRYHDWGATNARVGSLRRAVDFY', 'MFKVYGYDSNIHKCVYCDNAKRLLTVKKQPFEFINIMPEKGVFDDEKIAELLTKLGRDTQIGLTMPQVFAPDGSHIGGFDQLREYFK', 'APAFSVSPASGASDGQSVSVSVAAAGETYYIAQCAPVGGQDACNPATATSFTTDASGAASFSFTVRKSYAGQTPSGTPVGSVDCATDACNLGAGNSGLNLGHVALTFG', 'TPAFNKPKVELHVHLDGAIKPETILYFGKKRGIALPADTVEELRNIIGMDKPLSLPGFLAKFDYYMPVIAGCREAIKRIAYEFVEMKAKEGVVYVEVRYSPHLLANSKVDPMPWNQTEGDVTPDDVVDLVNQGLQEGEQAFGIKVRSILCCMRHQPSWSLEVLELCKKYNQKTVVAMDLAGDETIEGSSLFPGHVEAYEGAVKNGIHRTVHAGEVGSPEVVREAVDILKTERVGHGYHTIEDEALYNRLLKENMHFEVCPWSSYLTGAWDPKTTHAVVRFKNDKANYSLNTDDPLIFKSTLDTDYQMTKKDMGFTEEEFKRLNINAAKSSFLPEEEKKELLERLYREYQ', 'GNNVVVLGTQWGDEGKGKIVDLLTERAKYVVRYQGGHNAGHTLVINGEKTVLHLIPSGILRENVTSIIGNGVVLSP

'\nhmm_data is a list of pairs, where pair[0] is the observed sequence and pair[1] is the hidden sequence\n'

In [86]:
'''
Baum-Welch Algorithm Version 1
'''
# #Baum-Welch Implementation
# def initialize_hmm(states, observations):
#     # Initialize random probabilities for A, E, and π
#     n_states = len(states)
#     n_obs = len(observations)
    
#     A = np.random.dirichlet(np.ones(n_states), size=n_states)  # Transition probabilities
#     E = np.random.dirichlet(np.ones(n_obs), size=n_states)    # Emission probabilities
#     I = np.random.dirichlet(np.ones(n_states))               # Initial probabilities
#     return A, E, I

# def forward(obs_seq, states, A, E, I):
#     n_states = len(states)
#     T = len(obs_seq)
#     alpha = np.zeros((T, n_states))
    
#     # Initialization
#     alpha[0, :] = I * E[:, obs_seq[0]]
    
#     # Induction
#     for t in range(1, T):
#         for j in range(n_states):
#             alpha[t, j] = np.sum(alpha[t-1, :] * A[:, j]) * E[j, obs_seq[t]]
    
#     return alpha

# def backward(obs_seq, states, A, E):
#     n_states = len(states)
#     T = len(obs_seq)
#     beta = np.zeros((T, n_states))
    
#     # Initialization
#     beta[T-1, :] = 1
    
#     # Induction
#     for t in range(T-2, -1, -1):
#         for i in range(n_states):
#             beta[t, i] = np.sum(A[i, :] * E[:, obs_seq[t+1]] * beta[t+1, :])
    
#     return beta

# def baum_welch(data, states, observations):
#     max_iter = 100
#     tol = 1e-4
#     n_states = len(states)
#     n_obs = len(observations)
#     A, E, I = initialize_hmm(states, observations)
    
#     for iteration in range(max_iter):
#         A_new = np.zeros_like(A)
#         E_new = np.zeros_like(E)
#         pi_new = np.zeros_like(pi)
        
#         log_likelihood = 0
        
#         for obs_seq in data:
#             T = len(obs_seq)
#             alpha = forward(obs_seq, states, A, E, pi)
#             beta = backward(obs_seq, states, A, E)
            
#             # Compute gamma and xi
#             gamma = (alpha * beta) / np.sum(alpha[-1, :])  # State probabilities
#             xi = np.zeros((T-1, n_states, n_states))       # Transition probabilities
            
#             for t in range(T-1):
#                 denom = np.sum(alpha[t, :] * A * E[:, obs_seq[t+1]] * beta[t+1, :])
#                 for i in range(n_states):
#                     xi[t, i, :] = alpha[t, i] * A[i, :] * E[:, obs_seq[t+1]] * beta[t+1, :] / denom
            
#             # Update initial probabilities
#             pi_new += gamma[0, :]
            
#             # Update transition probabilities
#             for t in range(T-1):
#                 A_new += xi[t, :, :]
            
#             # Update emission probabilities
#             for t in range(T):
#                 E_new[:, obs_seq[t]] += gamma[t, :]
            
#             # Accumulate log likelihood
#             log_likelihood += np.log(np.sum(alpha[-1, :]))
        
#         # Normalize to get probabilities
#         pi = pi_new / np.sum(pi_new)
#         A = A_new / np.sum(A_new, axis=1, keepdims=True)
#         E = E_new / np.sum(E_new, axis=1, keepdims=True)
        
#         print(f"Iteration {iteration}, Log Likelihood: {log_likelihood}")
        
#         # Check for convergence
#         if iteration > 0 and abs(log_likelihood - prev_log_likelihood) < tol:
#             break
        
#         prev_log_likelihood = log_likelihood
    
#     return A, E, I

'\nBaum-Welch Algorithm Version 1\n'

In [141]:
'''
Baum-Welch Version 2
'''
from collections import defaultdict

smoothing = 1e-3

# Initialize the HMM parameters
def initialize_hmm(states, obs_vocab):
    A = {s1: {s2: np.random.rand() for s2 in states} for s1 in states}
    E = {s: {o: np.random.rand() for o in obs_vocab} for s in states}
    pi = {s: np.random.rand() for s in states}

    for s1 in A:
        total = sum(A[s1].values()) + len(A[s1]) * smoothing
        for s2 in A[s1]:
            A[s1][s2] = (A[s1][s2] + smoothing) / total
    for s in E:
        total = sum(E[s].values()) + len(E[s]) * smoothing
        for o in E[s]:
            E[s][o] = (E[s][o] + smoothing) / total
    total = sum(pi.values()) + len(pi) * smoothing
    for s in pi:
        pi[s] = (pi[s] + smoothing) / total

    return A, E, pi

# Forward algorithm (logarithmic scaling)
def forward(obs_seq, states, A, E, pi):
    
    n = len(obs_seq)
    alpha = np.full((n, len(states)), float('-inf'))  # Use log probabilities
    state_idx = {state: i for i, state in enumerate(states)}

    # Initialization
    for state in states:
        alpha[0][state_idx[state]] = np.log(pi[state]) + np.log(E[state].get(obs_seq[0], smoothing))

    # Recursion
    for t in range(1, n):
        for curr_state in states:
            log_probs = [
                alpha[t-1][state_idx[prev_state]] + np.log(A[prev_state].get(curr_state, smoothing))
                for prev_state in states
            ]
            alpha[t][state_idx[curr_state]] = np.log(E[curr_state].get(obs_seq[t], smoothing)) + log_sum_exp(log_probs)

    return alpha

# Backward algorithm (logarithmic scaling)
def backward(obs_seq, states, A, E):
    
    n = len(obs_seq)
    beta = np.full((n, len(states)), float('-inf'))
    state_idx = {state: i for i, state in enumerate(states)}

    # Initialization
    for state in states:
        beta[n-1][state_idx[state]] = 0  # log(1) = 0

    # Recursion
    for t in range(n-2, -1, -1):
        for curr_state in states:
            log_probs = [
                beta[t+1][state_idx[next_state]] +
                np.log(A[curr_state].get(next_state, smoothing)) +
                np.log(E[next_state].get(obs_seq[t+1], smoothing))
                for next_state in states
            ]
            beta[t][state_idx[curr_state]] = log_sum_exp(log_probs)

    return beta

# Compute log(sum(exp(log_probs))) safely
def log_sum_exp(log_probs):
    max_log = max(log_probs)
    if max_log == float('-inf'):
        return float('-inf')
    return max_log + np.log(sum(np.exp(lp - max_log) for lp in log_probs))

# Baum-Welch algorithm for parameter estimation
def baum_welch(hmm_data, states, obs_vocab, max_iter, tol):
    print("baum-welch called")
    A, E, pi = initialize_hmm(states, obs_vocab)

    for iteration in range(max_iter):
        print("iteration: ", iteration)
        A_num = defaultdict(lambda: defaultdict(float))
        A_den = defaultdict(float)
        E_num = defaultdict(lambda: defaultdict(float))
        E_den = defaultdict(float)
        pi_num = defaultdict(float)

        total_log_likelihood = 0
        # count = 0
        for obs_seq, _ in hmm_data:
            # print('count: ', count)
            # count +=1
            n = len(obs_seq)
            alpha = forward(obs_seq, states, A, E, pi)
            beta = backward(obs_seq, states, A, E)
            # print("alpha: ", alpha, " beta: ", beta)

            gamma = np.zeros((n, len(states)))
            xi = np.zeros((n-1, len(states), len(states)))
            state_idx = {state: i for i, state in enumerate(states)}

            for t in range(n):
                denom = log_sum_exp([alpha[t][state_idx[s]] + beta[t][state_idx[s]] for s in states])
                for state in states:
                    gamma[t][state_idx[state]] = alpha[t][state_idx[state]] + beta[t][state_idx[state]] - denom

            for t in range(n-1):
                denom = log_sum_exp([
                    alpha[t][state_idx[s1]] +
                    np.log(A[s1].get(s2, smoothing)) +
                    np.log(E[s2].get(obs_seq[t+1], smoothing)) +
                    beta[t+1][state_idx[s2]]
                    for s1 in states for s2 in states
                ])
                for s1 in states:
                    for s2 in states:
                        xi[t][state_idx[s1]][state_idx[s2]] = (
                            alpha[t][state_idx[s1]] +
                            np.log(A[s1].get(s2, smoothing)) +
                            np.log(E[s2].get(obs_seq[t+1], smoothing)) +
                            beta[t+1][state_idx[s2]] - denom
                        )

            total_log_likelihood += log_sum_exp(alpha[-1])
            # print('total_log_likelihood: ', total_log_likelihood)

            for s1 in states:
                pi_num[s1] += np.exp(gamma[0][state_idx[s1]])
                for t in range(n-1):
                    A_den[s1] += np.exp(gamma[t][state_idx[s1]])
                    for s2 in states:
                        A_num[s1][s2] += np.exp(xi[t][state_idx[s1]][state_idx[s2]])
                for t in range(n):
                    E_den[s1] += np.exp(gamma[t][state_idx[s1]])
                    E_num[s1][obs_seq[t]] += np.exp(gamma[t][state_idx[s1]])

        for s1 in states:
            pi[s1] = pi_num[s1] / len(hmm_data)
            for s2 in states:
                A[s1][s2] = A_num[s1][s2] / (A_den[s1] + smoothing)
            for obs in obs_vocab:
                E[s1][obs] = E_num[s1][obs] / (E_den[s1] + smoothing)

        if iteration > 0 and abs(total_log_likelihood - prev_log_likelihood) < tol:
            break
        prev_log_likelihood = total_log_likelihood

    return A, E, pi

In [110]:
# # Create a mapping for observations
# states = ['H', 'E', 'C']
# unique_observations = set(''.join(observed_states))  # Get unique characters
# obs_mapping = {char: idx for idx, char in enumerate(unique_observations)}

# # Convert observed states to integer sequences
# integer_observed_states = [
#     [obs_mapping[char] for char in seq]
#     for seq in observed_states
# ]

# # Update hmm_data with integer sequences
# hmm_data = list(zip(integer_observed_states, hidden_states))



# A, E, I = baum_welch(hmm_data, states, list(range(len(obs_mapping))))  # Observations are now indexed

# print("pi shape:", I.shape)          # Should be (3,)
# print("E shape:", E.shape)            # Should be (3, number_of_observations)
# print("obs_seq:", observed_states)            # Should be a list of integers
# print("obs_seq[0]:", observed_states[0])      # Should be an integer between 0 and number_of_observations - 1
# print("E[:, obs_seq[0]] shape:", E[:, observed_states[0]].shape)  # Should be (3,)


#Compute Initial, Emission, Transition log Probabilities

In [88]:
# from collections import Counter

# def compute_initial_probs(data):

#     # Count occurrences of initial hidden states
#     initial_counts = Counter(hid_seq[0] for _, hid_seq in data if hid_seq)

#     # Get all possible hidden states dynamically
#     hidden_states = set(initial_counts.keys())
#     total_initials = sum(initial_counts.values())
    
#     # Compute initial probabilities with smoothing
#     num_hidden_states = len(hidden_states)
#     initial_probs = {state: (initial_counts[state] + 1) / (total_initials + num_hidden_states)
#                      for state in hidden_states}
    
#     # Add unobserved states with uniform probability
#     for state in ['H', 'E', 'C']:
#         if state not in initial_probs:
#             initial_probs[state] = 1 / (total_initials + num_hidden_states)
    
#     return initial_probs

# initial_probs = compute_initial_probs(hmm_data)

# def compute_emission_probs(data):
#     from collections import Counter

#     # Collect all possible observations and hidden states
#     observations = set()
#     hidden_states = set()
#     for obs_seq, hid_seq in data:
#         observations.update(obs_seq)
#         hidden_states.update(hid_seq)

#     # Initialize counts
#     emission_counts = {hid: Counter() for hid in hidden_states}
#     hidden_counts = Counter()

#     # Count occurrences
#     for obs_seq, hid_seq in data:
#         for obs, hid in zip(obs_seq, hid_seq):
#             emission_counts[hid][obs] += 1
#             hidden_counts[hid] += 1

#     # Number of possible observations
#     V = len(observations)

#     # Compute emission probabilities with add-one smoothing
#     emission_probs = {}
#     for hid in hidden_states:
#         total = hidden_counts[hid] + V
#         emission_probs[hid] = {obs: (emission_counts[hid][obs] + 1) / total for obs in observations}
#     return emission_probs

# emission_probs = compute_emission_probs(hmm_data)


# def compute_transition_probs(data):
#     from collections import Counter

#     # Collect all possible hidden states
#     hidden_states = set()
#     for _, hid_seq in data:
#         hidden_states.update(hid_seq)

#     # Initialize counts
#     transition_counts = {hid: Counter() for hid in hidden_states}
#     total_transitions = Counter()

#     # Count occurrences
#     for _, hid_seq in data:
#         for i in range(1, len(hid_seq)):
#             prev = hid_seq[i - 1]
#             curr = hid_seq[i]
#             transition_counts[prev][curr] += 1
#             total_transitions[prev] += 1

#     # Number of possible hidden states
#     N = len(hidden_states)

#     # Compute transition probabilities with add-one smoothing
#     transition_probs = {}
#     for prev in hidden_states:
#         total = total_transitions[prev] + N
#         transition_probs[prev] = {curr: (transition_counts[prev][curr] + 1) / total for curr in hidden_states}
#     return transition_probs

# transition_probs = compute_transition_probs(hmm_data)

# print(initial_probs)
# print(emission_probs)
# print(transition_probs)



{'C': 1.0, 'H': 0.001953125, 'E': 0.001953125}
{'E': {'K': 0.047458051590283, 'T': 0.07272101177059855, 'C': 0.01828199348860506, 'M': 0.02153769095917856, 'H': 0.023415977961432508, 'X': 0.00025043826696719256, 'I': 0.09682569496619084, 'D': 0.03277610818933133, 'L': 0.09281868269471576, 'U': 3.130478337089907e-05, 'N': 0.030334335086401203, 'G': 0.0540633608815427, 'W': 0.019095917856248434, 'P': 0.021850738792887552, 'Z': 3.130478337089907e-05, 'E': 0.04357625845229151, 'A': 0.06492612071124468, 'Y': 0.051621587778612574, 'S': 0.05018156774355122, 'V': 0.12913223140495866, 'Q': 0.029551715502128727, 'R': 0.043858001502629605, 'F': 0.055659904833458555}, 'C': {'K': 0.05734704549416071, 'T': 0.06458871440569192, 'C': 0.013786109306416086, 'M': 0.017811019379783542, 'H': 0.02462484351973632, 'X': 0.0007606129272505428, 'I': 0.033546199312279146, 'D': 0.0786442074571759, 'L': 0.057061815646441756, 'U': 3.1692205302105944e-05, 'N': 0.06408163912085822, 'G': 0.1263568225394964, 'W': 0.011

#Implement Viterbi Algorithm

In [113]:
# def viterbi(obs_seq, states, initial_probs, emission_probs, transition_probs):
#     # Length of the observed sequence
#     n = len(obs_seq)

#     # Initialize Viterbi tables for probabilities and backtracking
#     v = {state: [float('-inf')] * n for state in states}
#     backpointer = {state: [None] * n for state in states}

#     # Initialization: Calculate the probability for the first observation
#     for state in states:
#         v[state][0] = np.log(initial_probs.get(state, 1e-3)) + np.log(emission_probs[state].get(obs_seq[0], 1e-3))
#         backpointer[state][0] = None  # No backtracking for the first observation

#     # Recursion: Fill in the Viterbi table for all subsequent observations
#     for t in range(1, n):
#         for curr_state in states:
#             # Compute the maximum probability and corresponding previous state
#             max_prob = float('-inf')
#             best_prev_state = None

#             for prev_state in states:
#                 transition_prob = v[prev_state][t-1] + \
#                                   np.log(transition_probs[prev_state].get(curr_state, 1e-3))
#                 if transition_prob > max_prob:
#                     max_prob = transition_prob
#                     best_prev_state = prev_state

#             # Update the Viterbi table and backpointer for the current state
#             v[curr_state][t] = max_prob + np.log(emission_probs[curr_state].get(obs_seq[t], 1e-3))
#             backpointer[curr_state][t] = best_prev_state

#     # Termination: Find the state with the maximum probability in the final column
#     max_final_state = None
#     max_final_prob = float('-inf')
#     for state in states:
#         if v[state][-1] > max_final_prob:
#             max_final_prob = v[state][-1]
#             max_final_state = state

#     # Backtracking: Reconstruct the most likely path
#     best_path = []
#     current_state = max_final_state

#     # Append states to the end of best_path during backtracking
#     for t in range(n-1, 0, -1):
#         best_path.append(current_state)
#         current_state = backpointer[current_state][t]

#     # Add the initial state (t = 0)
#     best_path.append(current_state)

#     # Reverse the list to get the correct order
#     best_path.reverse()

#     return best_path, max_final_prob

def viterbi(obs_seq, states, pi, E, A):
    """
    Parameters:
    - obs_seq: Observed sequence (list or string)
    - states: List of hidden states
    - pi: Initial state probabilities (dict {state: prob})
    - E: Emission probabilities (dict {state: {obs: prob}})
    - A: Transition probabilities (dict {state1: {state2: prob}})
    """
    # Length of the observed sequence
    n = len(obs_seq)

    # Initialize Viterbi tables for probabilities and backtracking
    v = {state: [float('-inf')] * n for state in states}
    backpointer = {state: [None] * n for state in states}

    # Initialization: Calculate the probability for the first observation
    for state in states:
        v[state][0] = np.log(pi.get(state, 1e-3)) + np.log(E[state].get(obs_seq[0], 1e-3))
        backpointer[state][0] = None  # No backtracking for the first observation

    # Recursion: Fill in the Viterbi table for all subsequent observations
    for t in range(1, n):
        for curr_state in states:
            # Compute the maximum probability and corresponding previous state
            max_prob = float('-inf')
            best_prev_state = None

            for prev_state in states:
                transition_prob = v[prev_state][t-1] + np.log(A[prev_state].get(curr_state, 1e-3))
                if transition_prob > max_prob:
                    max_prob = transition_prob
                    best_prev_state = prev_state

            # Update the Viterbi table and backpointer for the current state
            v[curr_state][t] = max_prob + np.log(E[curr_state].get(obs_seq[t], 1e-3))
            backpointer[curr_state][t] = best_prev_state

    # Termination: Find the state with the maximum probability in the final column
    max_final_state = None
    max_final_prob = float('-inf')
    for state in states:
        if v[state][-1] > max_final_prob:
            max_final_prob = v[state][-1]
            max_final_state = state

    # Backtracking: Reconstruct the most likely path
    best_path = []
    current_state = max_final_state

    # Append states to the end of best_path during backtracking
    for t in range(n-1, 0, -1):
        best_path.append(current_state)
        current_state = backpointer[current_state][t]

    # Add the initial state (t = 0)
    best_path.append(current_state)

    # Reverse the list to get the correct order
    best_path.reverse()

    return best_path, max_final_prob


#Model Evaluation

In [145]:
def calculate_accuracy(true_seq, pred_seq):
    assert len(true_seq) == len(pred_seq), "True and predicted sequences must have the same length."
    matches = sum(1 for true, pred in zip(true_seq, pred_seq) if true == pred)
    accuracy = matches / len(true_seq) * 100
    return accuracy

def calculate_error_percentage(true_seq, pred_seq):
    assert len(true_seq) == len(pred_seq), "True and predicted sequences must have the same length."
    mismatches = sum(1 for true, pred in zip(true_seq, pred_seq) if true != pred)
    error_percentage = mismatches / len(true_seq) * 100
    return error_percentage


states = ['H', 'E', 'C']
total_accuracy = 0
total_error = 0
n_sequences = len(hmm_data[:300]) 
# initial_probs = compute_initial_probs(hmm_data)
# emission_probs = compute_emission_probs(hmm_data)
# transition_probs = compute_transition_probs(hmm_data)
obs_vocab = set(symbol for seq, _ in hmm_data for symbol in seq)
A, E, pi = baum_welch(hmm_data, states, obs_vocab, 60, 1e-4)


for obs_seq, true_hid_seq in hmm_data[:300]:
    
    pred_hid_seq, prob = viterbi(obs_seq, states , pi, E, A)
    
    accuracy = calculate_accuracy(true_hid_seq, pred_hid_seq)
    error_percentage = calculate_error_percentage(true_hid_seq, pred_hid_seq)

    print(f"True: {true_hid_seq}")
    print(f"Pred: {''.join(pred_hid_seq)}")
    print(f"Log Probability: {prob}")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Error Percentage: {error_percentage:.2f}%\n")

    total_accuracy += accuracy
    total_error += error_percentage

average_accuracy = total_accuracy / n_sequences
average_error = total_error / n_sequences
print(f"Average Accuracy: {average_accuracy:.2f}%")
print(f"Average Error Percentage: {average_error:.2f}%")

baum-welch called
iteration:  0
iteration:  1
iteration:  2


KeyboardInterrupt: 

In [91]:
train_data = hmm_data[:400]  # 80% training
test_data = hmm_data[400:]  # 20% testing

# initial_probs = compute_initial_probs(train_data)
# emission_probs = compute_emission_probs(train_data)
# transition_probs = compute_transition_probs(train_data)
total_accuracy = 0
total_error_percentage = 0
n_sequences = len(hmm_data[400:]) 
for obs_seq, true_hid_seq in test_data:
    pred_hid_seq, prob = viterbi(obs_seq, states, initial_probs, emission_probs, transition_probs)
    # Evaluate accuracy on the test set
    accuracy = calculate_accuracy(true_hid_seq, pred_hid_seq)
    error_percentage = calculate_error_percentage(true_hid_seq, pred_hid_seq)
    total_accuracy += accuracy
    total_error_percentage += error_percentage

print("total_accuracy: ", total_accuracy/n_sequences)
print("total_error: ",  total_error_percentage/n_sequences)
    



total_accuracy:  49.18552290019113
total_error:  50.81447709980887
